In [28]:
import pandas as pd
import json
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import xgboost as xgb


csv = pd.read_csv("data/hack_train.csv")
csvFull = csv.rename(columns={'text': 'answers', "label" : "is_human"})
csv = csvFull.drop(columns="src")
answers_df = csv

answers_df = answers_df.explode('answers', ignore_index=True)
answers_df = answers_df.dropna(subset=['answers'], ignore_index=True)

In [29]:
import nltk
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')
nltk.download('stopwords')
stop_words = stopwords.words('english')
wnl = WordNetLemmatizer()

def preprocess(text_column):
    new_review = []
    for review in text_column:
        text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(review).lower()).strip()
        text = [wnl.lemmatize(i) for i in text.split(' ') if i not in stop_words]
        new_review.append(' '.join(text))
    return new_review

answers_df["answers"] = preprocess(answers_df["answers"])

[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'stopwords' is not defined

In [49]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

X = answers_df["answers"]
y = answers_df["is_human"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cv = TfidfVectorizer(max_features=10000)
cv.fit(answers_df["answers"])
X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [52]:
import multiprocessing
from sklearn.model_selection import GridSearchCV


model = xgb.XGBClassifier(max_depth=6, n_estimators=2000, colsample_bytree=0.7, reg_alpha=0.01, reg_lambda=0.1)

"""
param_grid = {
    "colsample_bytree": [0.5, 0.7],
    "reg_alpha": [0.01, 0.1, 1.0],
    "reg_lambda": [0.01, 0.1, 1.0],
}

model = GridSearchCV(
        model,
        param_grid,
        verbose=1,
        n_jobs=4,
    )
"""


'\nparam_grid = {\n    "colsample_bytree": [0.5, 0.7],\n    "reg_alpha": [0.01, 0.1, 1.0],\n    "reg_lambda": [0.01, 0.1, 1.0],\n}\n\nmodel = GridSearchCV(\n        model,\n        param_grid,\n        verbose=1,\n        n_jobs=4,\n    )\n'

In [53]:
model.fit(X_train, y_train, verbose=1)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)
#print(model.best_params_)

0.8788278775079198

In [48]:
import pickle

filename = "XGB_91.pickle"
pickle.dump(model, open(filename, "wb"))

In [12]:
#print(model.best_params_)
test = cv.transform(["The oldest town i know is in the north of france i think"])
model.predict(test)

array([0])

In [9]:
with open('model/XGB_90.pickle', 'rb') as file:
    loaded_model = pickle.load(file)

In [13]:
loaded_model.predict(cv.transform(["The title of the oldest continuously inhabited town is often attributed to Jericho, located in the West Bank of the Palestinian Territories, with archaeological evidence suggesting settlement dating back over 11,000 years."]))

array([0])